# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Откроем файл с данными и изучим их, для этого подключим библиотеку `pandas` помимо неё подключим еще остальные библиотеки, которые нам пригодятся. Для того чтобы прочитать данные из датасета воспользуемся методом `read_csv`, для получения общей информации о датасете воспользуемся методом `info`.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except:
    data = pd.read_csv('Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


По данным видно, что есть пропуски в столбце `Tenure` — *сколько лет человек является клиентом банка*, далее проанализируем данные в этом столбце и попробуем заполнить пропуски, в остальных столбцах пропусков в данных нет. Также можно заметить, что в датасете в основом числовые данные, только три столбца с типом **object**, далее детальнее смотрим данные в этих в столбцах, но для начала приведем в корректный вид названия столбцов согласно стандарту Python.

Так как нам часто нужно приводить название столбцов в формат по стандарту, напишем небольшую функцию, которая будет переименовать столбцы в датасете, в дальнейшем её можно будет использовать в других проектах:

In [4]:
def rename_columns(columns): #функция для переименования столбцов
    '''
    Функция приводит название столбцов в датасете в формат shake case по стандарту Python,
    на вход функция принимает текущие названия столбцов в датасете в виде списка
    '''
    columns_rename = []
    new_name_column = ""
    for column in columns:
        for index_symbol in range(0, len(column), 1):
            if index_symbol == 0 and column[index_symbol].istitle(): #метод istitle проверяет является ли символ заглавным
                new_name_column += column[index_symbol].lower()
            elif index_symbol > 0 and column[index_symbol].istitle():
                new_name_column += '_' + column[index_symbol].lower()
            else:
                new_name_column += column[index_symbol]
        columns_rename.append(new_name_column)
        new_name_column = ""
    return columns_rename

In [5]:
data.columns = rename_columns(data.columns)
data.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')

Стобцы переименовались корректно, можно переходить к обработке пропусков. 

Как мы помним в датасете есть пропуски всего в одном столбце — `tenure` — *сколько лет человек является клиентом банка*, посмотрим для начала какие значения содержатся в этом столбце:

In [6]:
sorted(data['tenure'].unique())

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, nan]

Значения в этом столбце находятся в диапазоне от **0** до **10**, при значении **0** скорее всего имеется в виду, что человек был клиентом банка меньше одного года. Мы можем попробовать заполнить пропуски средним значением, но тогда у нас будет перекос в данных в сторону среднего значения, что может не соотвествовать реальности. Попробуем заполнить пропуски медианным значением, но для начала нам нужно найти столбец, который коррелирует со столбцом `tenure`, чтобы корректно рассчитать медиану для разных групп клиентов, потому что, например, клиенты разных возрастов могут быть клиентами банка разное количество лет, соответственно и медиана у них будет разная.

In [7]:
data.corr()['tenure']

row_number         -0.007322
customer_id        -0.021418
credit_score       -0.000062
age                -0.013134
tenure              1.000000
balance            -0.007911
num_of_products     0.011979
has_cr_card         0.027232
is_active_member   -0.032178
estimated_salary    0.010520
exited             -0.016761
Name: tenure, dtype: float64

Так как корреляция по всем столбцам почти равну нуля, то столбец `tenure` не корреляриует не с одним столбцом, либо эта корреляция очень слабая. Так как нам нужно для решения задачи обучить модель машинного обучения и пропусков где-то **10%**, целесобразнее будет удалить строки, где есть пропуски в столбце `tenure`, чтобы не испортить обучение модели машинного обучения.

In [8]:
data = data.loc[~(data['tenure'].isna())]

In [9]:
data.isna().sum()

row_number          0
customer_id         0
surname             0
credit_score        0
geography           0
gender              0
age                 0
tenure              0
balance             0
num_of_products     0
has_cr_card         0
is_active_member    0
estimated_salary    0
exited              0
dtype: int64

Пропусков в данных больше нет, проверим теперь есть ли у нас дубликаты в данных:

In [10]:
data.duplicated().sum()

0

Явных дубликатов нет, проверим теперь есть ли у нас неявные дубликаты, начнем со столбца `surname`— *фамилия*, проверим какие есть уникальные значения в этом столбце. Так как фамилий может быть большое количество, то сначала посчитаем количество уникальных фамилий, так как они выглядят в датасете, а потом приведем их к нижнему регистру и посчитаем еще раз количество уникальных фамилий:

In [11]:
print('Количество уникальных фамилий', data['surname'].nunique())
print('Количество уникальных фамилий в нижнем регистре', data['surname'].str.lower().nunique())

Количество уникальных фамилий 2787
Количество уникальных фамилий в нижнем регистре 2786


В столбце с фамилиями оказалось одно значение, которое записано в разных регистрах, преобразуем все значения в столбце с фамилиями в нижний регистр, чтобы избавиться от неявного дубликата:

In [12]:
data['surname'] = data['surname'].str.lower()

Посмотрим теперь какие уникальные значения есть в столбцах `geography` — *страна проживания* и `gender`— *пол*:

In [13]:
print(data['geography'].unique())
print(data['gender'].unique())

['France' 'Spain' 'Germany']
['Female' 'Male']


Дубликатов в значениях нет, эти столбцы можно не изменять, мы убедились, что нет одних и тех же значений в разном регистре. Проверять дубликаты в паре столбцов, например, фамилия и страна или фамилия и пол, не будем, так как в одной или в нескольких странах люди могут быть с одинаковыми фамилиями и полом.
В остальных столбцах обработку дубликатов делать не будем, так как они числовые и там естественно присутствие дублирующих значений.

Мы сделали подготовку данных можем теперь переходить к обучению моделей для решения нашей задачи.

## Исследование задачи

Прежде, чем разделить данные на выборки, уберем из датасета те столбцы, которые нам не пригодятся в обучении модели — это столбцы:
- `row_number` — *индекс строки в данных*;
- `customer_id` — *уникальный идентификатор клиента*;
- `surname` — *фамилия*.

Первые два столбца можно сказать, что они технические, не несут никакой смысловой нагрузки и не дают о человеке никакой информации как о клиенте банка. Со столбцом с фамилиями, точно такая ситуация, это конечно не технический столбец, в котором перечисляют порядковый номер строк, он нам предоставляют некую информацию о человеке, но он нам не предоставляет информацию об этом человеке как о клиенте банка, поэтому этот столбец мы тоже убираем из данных: 

In [14]:
data = data.drop(['row_number', 'customer_id', 'surname'], axis=1)
data.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Так как у нас есть категориальные признаки, то необходимо их преобразовать в числовые, для этого воспользуемся техникой **прямого кодирования**, то есть **OHE**. Чтобы применить **OHE** на наших данных, воспользуемся методом `get_dummies` из библиотеки `pandas`, при вызове метода для аргумента **drop_first** укажем значение **True**, чтобы не создавать лишние категории и не угодить в ловушку фиктивных признаков. Метод будет использовать только на двух столбцах с категориальными признаками `gender` — *пол* и `geography` — *страна проживания*, на остальных столбцах метод использовать не будем, так как в этих столбцах признаки количественные, либо уже обрабротаны техникой **прямого кодирования**, например как столбец `exited`, который содержит следующие значения (1 — клиент ушел, 0 — клиент остался).

In [15]:
data_ohe = pd.get_dummies(data, columns=['gender', 'geography'], drop_first=True)

Предобработку данных сделали, лишние столбцы убрали, категориальные признаки преобразовали, можно переходить к разделению данных на выборки для обучения моделей машинного обучения. Разобьем данные на три выборки — обучащая **60%**, валидационная **20%** и тестовая **20%**, для этого воспользуемся методом `train_test_split` из библиотеки `sklearn`.

In [16]:
data_train, data_valid = train_test_split(data_ohe, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_valid, test_size=0.5, random_state=12345)
print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)

(5454, 12)
(1818, 12)
(1819, 12)


По размерности видно, что выборки разделились, теперь выбирем из данных признаки и что необходимо предсказать. В задание указано, что нужно **построить модель, которая предскажет уйдет ли клиент из банка в ближайшее время или нет**. Факт ухода клиента указан в столбце `exited` (1 — клиент ушел, 0 — клиент остался), это будет целевой признак, который нам необходимо будет предсказать, а все остальные столбцы вынесем в признаки по которым будем предсказывать целевой признак.

In [17]:
data_train_features = data_train.drop(['exited'], axis=1)
data_train_target = data_train['exited']

data_valid_features = data_valid.drop(['exited'], axis=1)
data_valid_target = data_valid['exited']

data_test_features = data_test.drop(['exited'], axis=1)
data_test_target = data_test['exited']

Так как у нас данных присуствуют количественные признаки с разным разбросом значений, например, столбцы:
- `age` — возраст;
- `balance` — баланс на счёте.

Очевидно, что значения баланса баланса на счёте, может сильно превышать значения возраста клиента и соответстенно разброс значений у столбца с балансом счёта будет больше, чем у столбца с возрастом, в таком случае алгорим может решить, что признаки с большими значениями и разбросом важнее. Чтобы избежать этой ловушки, признаки масштабируются — приводятся к одному масштабу. Один из методов масштабирования — **стандартизации данных**, примением его на наших данных, для этого воспользуемся структурой *StandardScaler* из библиотеки `sklearn`.

In [18]:
scaler = StandardScaler()
columns_names = data_train_features.columns
scaler.fit(data_train_features[columns_names])

data_train_features[columns_names] = scaler.transform(data_train_features[columns_names])
data_valid_features[columns_names] = scaler.transform(data_valid_features[columns_names])
data_test_features[columns_names] = scaler.transform(data_test_features[columns_names])

Возьмем для решения нашей задачи несколько моделей машинного обучения — **Дерево решений**, **Случайный лес**, **Логистическая регрессия** и переберем в них гиперпараметры, чтобы найти лучшую модель для решения нашей задачи. Для оценок моделей будем использовать метрики *accuracy*, *AUC-ROC* и *F1*-меру.

Начнем с модели **Дерево решений**. 
Для начала напишем функцию, в которой будем перебирать гиперпараметры для поиска наилучшей модели, для определения лучшей модели будем использовать метрику *F1*-мера:

In [19]:
def get_best_model_tree(features, target, valid_features, valid_target, val_class_weight):
    '''
    Функция перебирает гиперпараметры у модели Дерево решений для поиска наилучшей модели
    '''
    best_result_model_tree = 0
    best_max_depth = 0
    best_model_tree = None
    
    for depth in range(1, 11):
        model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight=val_class_weight)
        model_tree.fit(data_train_features, data_train_target)
        predictions_tree = model_tree.predict(data_valid_features)
        result_tree = f1_score(data_valid_target, predictions_tree)

        if result_tree > best_result_model_tree:
            best_result_model_tree = result_tree
            best_max_depth = depth
            best_model_tree = model_tree

    print('best_max_depth', best_max_depth, 'f1_score:', best_result_model_tree)
    return best_model_tree

In [20]:
best_model_tree = get_best_model_tree(data_train_features, data_train_target, data_valid_features, data_valid_target, None)

best_max_depth 7 f1_score: 0.5773524720893142


Максимальное значение по метрике *F1*-мере достигается при глубине дерева равное **7** и равна **0.57**, проверим нашу модель на остальных метриках *accuracy* и *AUC-ROC*. Для начала напишем функцию, которая будет записывать полученные значения по метрикам в небольшую таблицу для того чтобы было проще их сравнивать между собой и модель у нас будет не одна. Также напишем отдельную функцию, которая будет рассчитывать различные метрики для оценки качества моделей:

In [21]:
def fill_metrics_score(metrics, index, model_name, val_accuracy_score, val_roc_auc_score, val_f1_score):
    '''
    Функция записывает полученные метрики по моделям в таблицу
    '''
    metrics.loc[index, 'model'] = model_name
    metrics.loc[index, 'accuracy'] = val_accuracy_score
    metrics.loc[index, 'roc_auc_score'] = val_roc_auc_score
    metrics.loc[index, 'f1_score'] = val_f1_score
    return metrics

In [22]:
def calc_metrics_model(model, model_name, features_valid, target_valid, metrics_info):
    '''
    Функция рассчитывает метрики у обученной модели и записывает их в таблицу
    '''
    predictions_model = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    metrics_info = fill_metrics_score(metrics_models, len(metrics_models), model_name,
                                   accuracy_score(target_valid, predictions_model),
                                   roc_auc_score(target_valid, probabilities_one_valid),
                                   f1_score(target_valid, predictions_model))
    
    return metrics_info

In [23]:
metrics_models = pd.DataFrame({'model': [], 'accuracy': [], 'roc_auc_score': [], 'f1_score': []})
metrics_models = calc_metrics_model(best_model_tree, 'DecisionTreeClassifier', data_valid_features, 
                                    data_valid_target, metrics_models)
metrics_models

,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352


Хоть метрика *accuracy* показала качество модели равное почти **0.85**, метрика *F1*-меры, которая включает в себя полноту и точность, показывает качество модели равное всего **0.57**, это сильно меньше чем у метрики *accuracy*.

Посмотрим теперь как с нашей задачей справится модель **Случайного леса:**
Как с моделью **Дерево решений**, напишем функцию по перебору гиперпараметров для модели **Случайный лес**:

In [24]:
def get_best_model_randomforest(features, target, valid_features, valid_target, val_class_weight):
    '''
    Функция перебирает гиперпараметры у модели Случайный лес для поиска наилучшей модели
    '''
    best_result_randomforest = 0
    best_max_depth = 0
    best_est = 0
    best_model_model_randomforest = None
    
    for depth in range(1, 11, 1):
        for est in range(10, 101, 10):
            model_randomforest = RandomForestClassifier(max_depth=depth, n_estimators=est, class_weight=val_class_weight)
            model_randomforest.fit(data_train_features, data_train_target)
            predictions_randomforest = model_randomforest.predict(data_valid_features)
            result_randomforest = f1_score(data_valid_target, predictions_randomforest)

            if result_randomforest > best_result_randomforest:
                best_result_randomforest = result_randomforest
                best_max_depth = depth
                best_est = est
                best_model_model_randomforest = model_randomforest

    print('best_est', best_est, 'best_max_depth', best_max_depth, 'f1_score:', best_result_randomforest)
    return best_model_model_randomforest

In [25]:
best_model_randomforest = get_best_model_randomforest(data_train_features, data_train_target, data_valid_features, 
                                                      data_valid_target, None)

best_est 20 best_max_depth 10 f1_score: 0.5854483925549915


Максимальное значение по метрике *F1*-меры достигается при количестве деревьев равное **20** и глубине деревьев равное **10** и равно **0.58**, проверим эту модель с такими гиперпараметрами с помощью метрик *AUC-ROC* и *accuracy*:

In [26]:
metrics_models = calc_metrics_model(best_model_randomforest, 'RandomForestClassifier', data_valid_features, 
                                    data_valid_target, metrics_models)
metrics_models

,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448


Модель **Случайного леса** по метрике *accuracy* показывает качество модели равное почти **0.86**, однако по метрике *F1*-мера качество модели равно почти **0.57**, качество сильно падает как и у модели **Дерево решений**.

Проверим ещё модель **Логистической регрессии**, может обучив её, мы получим оптимальное качество модели по всем метрикам:

In [27]:
model_logistic = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_logistic.fit(data_train_features, data_train_target)
predictions_logistic = model_logistic.predict(data_valid_features)

print('accuracy_score:', accuracy_score(data_valid_target, predictions_logistic))

accuracy_score: 0.8080308030803081


По метрике *accuracy* качество модели равно **0.80**, проверим теперь модель с помощью метрик *AUC-ROC* и *F1*-меры:

In [28]:
metrics_models = calc_metrics_model(model_logistic, 'LogisticRegression', data_valid_features, data_valid_target, 
                                    metrics_models)
metrics_models

,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393


Модели **Логистической регрессии** оказалась самой плохой по сравнению с другими моделями, хоть по метрике *accuracy* качество модели равно **0.80**, однако по метрике *F1*-мера качество очень сильно падает и равно всего **0.30**, данная модель предсказывает очень мало правильных ответов.

Плохие показатели по метрике *F1*-мера скорей всего связаны с тем, что у нас в данных не сбалисированные классы, а наши модели, которые мы обучили считают, что у нас все классы равнозначные между собой, даже те, которые встречаются редко.

## Борьба с дисбалансом

Для того чтобы избавить от проблемы дисбаланса классов попробуем их сбалансировать, как раз у тех моделей машинного обучения, чтобы мы обучили ранее, есть аргумент *class_weight*, если указать `class_weight='balanced'` алгоритм посчитает, во сколько раз один класс встречается чаще другого класса и присвоет больший вес редкому классу.

Обучим наши модели **Дерево решений**, **Случайный лес**, **Логистическая регрессия** с указанием аргумента `class_weight='balanced'` и посмотрим какие будут значения у метрик *accuracy*, *AUC-ROC* и *F1*-меры.

In [29]:
model_tree_class_weight = get_best_model_tree(data_train_features, data_train_target, data_valid_features, data_valid_target, 
                                              'balanced')
metrics_models = calc_metrics_model(model_tree_class_weight, 'DecisionTreeClassifier_class_weight', data_valid_features, 
                                    data_valid_target, metrics_models)
metrics_models

best_max_depth 5 f1_score: 0.5735449735449736


,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545


In [30]:
model_randomforest_class_weight = get_best_model_randomforest(data_train_features, data_train_target, data_valid_features, 
                                                              data_valid_target, 'balanced')
metrics_models = calc_metrics_model(model_randomforest_class_weight, 'RandomForestClassifier_class_weight', 
                                    data_valid_features, data_valid_target, metrics_models)
metrics_models

best_est 50 best_max_depth 9 f1_score: 0.6458072590738423


,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545
4,RandomForestClassifier_class_weight,0.844334,0.871136,0.645807


In [31]:
model_logistic_class_weight = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000, class_weight='balanced')
model_logistic_class_weight.fit(data_train_features, data_train_target)
metrics_models = calc_metrics_model(model_logistic_class_weight, 'LogisticRegression_class_weight', 
                                    data_valid_features, data_valid_target, metrics_models)
metrics_models

,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545
4,RandomForestClassifier_class_weight,0.844334,0.871136,0.645807
5,LogisticRegression_class_weight,0.709021,0.777753,0.509731


Как видно по всем моделям значение по метрике *accuracy* падает, особенно это заметно на модели **Логистической регрессии** с **0.80** упало до **0.70**. Значение метрики *F1*-меры выросло у всех моделей, у модели **Случайного леса** самый высокий показатель данной метрики — **0.65** выше значения **0.59** которое у нас указано в задании, возможно мы нашли модель, которая подходит для решения нашей задачи. Также и выросло значение по метрике *AUC-ROC*, это метрика показывает насколько наши модели отличается от случайной, *AUC-ROC* случайной модели равен **0.5**, у всех наших трех моделей это значений больше **0.5**, однако у **Логистической регрессии** значение по данной метрики равно **0.77**. Также у модели **Логистической регрессии** самое низкое значение по метрике *F1*-мера — **0.50**, но конечно сильно выросло по сравнению с модель без балансироки классов, однако этого недостаточно для решения нашего задания.

Попробуем сбалансировать классы путём увеличения их числа такая техникой **upsampling**. Данная техника представляет из себя то, что мы сначала определяем меньший класс, который содержит меньше объектов, и копируем несколько раз объекты этого класса, и создает новую выборку засчет полученных данных.

Напишем для начала функцию, которая будет применять технику **upsampling** на наших данных:

In [32]:
def upsampling(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

Функцию будем применять только на обучающей выборке, чтобы у модели было больше примеров для обучения:

In [33]:
print('Количество элементов с классом 0 до применения техники upsampling', 
      len(data_train_features[data_train_target == 0]),
      '\nКоличество элементов с классом 1 до применения техники upsampling', 
      len(data_train_features[data_train_target == 1])
     )
data_train_features_upsampled, data_train_target_upsampled = upsampling(data_train_features, data_train_target, 4)
print('Количество элементов с классом 0 после применения техники upsampling', 
      len(data_train_features_upsampled[data_train_target_upsampled == 0]),
      '\nКоличество элементов с классом 1 после применения техники upsampling', 
      len(data_train_features_upsampled[data_train_target_upsampled == 1])
     )

Количество элементов с классом 0 до применения техники upsampling 4328 
Количество элементов с классом 1 до применения техники upsampling 1126
Количество элементов с классом 0 после применения техники upsampling 4328 
Количество элементов с классом 1 после применения техники upsampling 4504


Как можно заметить выше, дислбанс классов мы устранили, получилось конечно не ровно пополам, но гораздо лучше, чем разница между элементами класса больше, чем 3 тысячи элементов.

Проверим теперь модели **Дерево решений**, **Случайный лес**, **Логистическая регрессия** на этих данных и посмотрим какие у них будут значения в метриках качества *accuracy*, *AUC-ROC* и *F1*-меры.

In [34]:
model_tree_upsampled = get_best_model_tree(data_train_features_upsampled, data_train_target_upsampled, 
                                           data_valid_features, data_valid_target, None)
metrics_models = calc_metrics_model(model_tree_upsampled, 'DecisionTreeClassifier_upsampled', 
                                    data_valid_features, data_valid_target, metrics_models)
metrics_models

best_max_depth 7 f1_score: 0.5773524720893142


,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545
4,RandomForestClassifier_class_weight,0.844334,0.871136,0.645807
5,LogisticRegression_class_weight,0.709021,0.777753,0.509731
6,DecisionTreeClassifier_upsampled,0.854235,0.835244,0.577352


In [35]:
model_randomforest_upsampled = get_best_model_randomforest(data_train_features_upsampled, data_train_target_upsampled, 
                                                           data_valid_features, data_valid_target, None)
metrics_models = calc_metrics_model(model_randomforest_upsampled, 'RandomForestClassifier_upsampled',  
                                    data_valid_features, data_valid_target, metrics_models)
metrics_models

best_est 80 best_max_depth 10 f1_score: 0.577319587628866


,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545
4,RandomForestClassifier_class_weight,0.844334,0.871136,0.645807
5,LogisticRegression_class_weight,0.709021,0.777753,0.509731
6,DecisionTreeClassifier_upsampled,0.854235,0.835244,0.577352
7,RandomForestClassifier_upsampled,0.864686,0.870085,0.577320


In [36]:
model_logistic_upsampled = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_logistic_upsampled.fit(data_train_features_upsampled, data_train_target_upsampled)
metrics_models = calc_metrics_model(model_logistic_upsampled, 'LogisticRegression_upsampled', 
                                    data_valid_features, data_valid_target, metrics_models)
metrics_models

,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545
4,RandomForestClassifier_class_weight,0.844334,0.871136,0.645807
5,LogisticRegression_class_weight,0.709021,0.777753,0.509731
6,DecisionTreeClassifier_upsampled,0.854235,0.835244,0.577352
7,RandomForestClassifier_upsampled,0.864686,0.870085,0.577320
8,LogisticRegression_upsampled,0.700770,0.777862,0.508137


Как видно по сравнению с первыми моделями, при использовании техники **upsampling** значение у метрики *accuracy* падает, у метрик *AUC-ROC* и *F1*-меры значения растут. Если смотреть по метрике *F1*-меры, она незначительно поменялась у модели **Дерево решений** и **Логистической регрессии** и упала у модели **Случайного леса** по сравнению с моделями, где сделали балансировку классов с помощью аргумента `class_weight`, значение метрик у данной модели упало с **0.64** до **0.57**. Если смотреть на метрику *AUC-ROC*, то в основном значение у метрики не сильно поменялось по сравнению с моделями, где сделали балансировку классов с помощью аргумента `class_weight`.

Попробуем теперь сбалансировать классы с помощью техники **downsampling**. Эта техника противоположна технике **upsampling**, в прошлый мы увеличивали количество значений меньшего класса, сейчас же будет делать наоборот — уменьшать количество значений большего класса случайным образом. Как и в прошлый раз напишем функцию для использования техники **downsampling**: 

In [37]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

Функцию будем применять только на обучающей выборке, чтобы у модели было больше примеров для обучения:

In [38]:
print('Количество элементов с классом 0 до применения техники downsample', 
      len(data_train_features[data_train_target == 0]),
      '\nКоличество элементов с классом 1 до применения техники downsample', 
      len(data_train_features[data_train_target == 1])
     )
data_train_features_downsample, data_train_target_downsample = downsample(data_train_features, data_train_target, 0.2601)
print('Количество элементов с классом 0 после применения техники downsample', 
      len(data_train_features_downsample[data_train_target_downsample == 0]),
      '\nКоличество элементов с классом 1 после применения техники downsample', 
      len(data_train_features_downsample[data_train_target_downsample == 1])
     )

Количество элементов с классом 0 до применения техники downsample 4328 
Количество элементов с классом 1 до применения техники downsample 1126
Количество элементов с классом 0 после применения техники downsample 1126 
Количество элементов с классом 1 после применения техники downsample 1126


Как можно заметить выше, дислбанс классов мы устранили, получилось ровно пополам элементов каждого класса.

Проверим теперь модели **Дерево решений**, **Случайный лес**, **Логистическая регрессия** на этих данных и посмотрим какие у них будут значения в метриках качества *accuracy*, *AUC-ROC* и *F1*-меры.

In [39]:
model_tree_downsample = get_best_model_tree(data_train_features_downsample, data_train_target_downsample, 
                                            data_valid_features, data_valid_target, None)
metrics_models = calc_metrics_model(model_tree_downsample, 'DecisionTreeClassifier_downsample', 
                                    data_valid_features, data_valid_target, metrics_models)
metrics_models

best_max_depth 7 f1_score: 0.5773524720893142


,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545
4,RandomForestClassifier_class_weight,0.844334,0.871136,0.645807
5,LogisticRegression_class_weight,0.709021,0.777753,0.509731
6,DecisionTreeClassifier_upsampled,0.854235,0.835244,0.577352
7,RandomForestClassifier_upsampled,0.864686,0.870085,0.577320
8,LogisticRegression_upsampled,0.700770,0.777862,0.508137
9,DecisionTreeClassifier_downsample,0.854235,0.835244,0.577352


In [40]:
model_randomforest_downsample = get_best_model_randomforest(data_train_features_upsampled, data_train_target_upsampled, 
                                                            data_valid_features, data_valid_target, None)
metrics_models = calc_metrics_model(model_randomforest_downsample, 'RandomForestClassifier_downsample', 
                                    data_valid_features, data_valid_target, metrics_models)
metrics_models

best_est 10 best_max_depth 9 f1_score: 0.5838926174496645


,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545
4,RandomForestClassifier_class_weight,0.844334,0.871136,0.645807
5,LogisticRegression_class_weight,0.709021,0.777753,0.509731
6,DecisionTreeClassifier_upsampled,0.854235,0.835244,0.577352
7,RandomForestClassifier_upsampled,0.864686,0.870085,0.577320
8,LogisticRegression_upsampled,0.700770,0.777862,0.508137
9,DecisionTreeClassifier_downsample,0.854235,0.835244,0.577352


In [41]:
model_logistic_downsample = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_logistic_downsample.fit(data_train_features_downsample, data_train_target_downsample)
metrics_models = calc_metrics_model(model_logistic_downsample, 'LogisticRegression_downsample', 
                                    data_valid_features, data_valid_target, metrics_models)
metrics_models

,model,accuracy,roc_auc_score,f1_score
0,DecisionTreeClassifier,0.854235,0.835244,0.577352
1,RandomForestClassifier,0.865237,0.857935,0.585448
2,LogisticRegression,0.808031,0.773617,0.303393
3,DecisionTreeClassifier_class_weight,0.778328,0.839652,0.573545
4,RandomForestClassifier_class_weight,0.844334,0.871136,0.645807
5,LogisticRegression_class_weight,0.709021,0.777753,0.509731
6,DecisionTreeClassifier_upsampled,0.854235,0.835244,0.577352
7,RandomForestClassifier_upsampled,0.864686,0.870085,0.577320
8,LogisticRegression_upsampled,0.700770,0.777862,0.508137
9,DecisionTreeClassifier_downsample,0.854235,0.835244,0.577352


Модели, где мы сбалансировали классы с помощью техники **downsample** показывают почти точно такую же точность по метрике *F1*-меры по сравнению с другими моделями, где мы использовали другие методы балансировки классов, модель **Случайного леса**, которая показывала до этого наибольшее значение по метрике *F1*-меры, сейчас имеет значение **0.58**. По метрике *AUC-ROC* видно, что значения не сильно изменились по сравнению с остальными моделями, где мы балансировали классы с помощью аргумента `class_weight` и техники **upsampled**.

На основании выше проведенного исследования, мы нашли модель, которая подходит для решения нашей задачи — это модель **Случайного леса** с гиперпараметрами с гиперпараметрами `n_estimators` — количество деревьев в лесу равное **50** и `max_depth` — глубина дерева равное **9**.Также мы нашли подходящий способ балансировки классов — это балансировака классов с помощью аргумента `class_weight`. При учите выше перечисленного данная модель **Случайного леса** показывает качество равное **0.64** по метркие *F1*-мера.

## Тестирование модели

Ранее нам удалось определить, что лучшую точность при обучении показала модель **Случайный лес** с гиперпараметрами `n_estimators` — количество деревьев в лесу равное **50** и `max_depth` — глубина дерева равное **9** при балансировке классов с помощью аргумента `class_weight`. Проверим теперь выбранную нами модель на тестовых данных, данная модель у нас записана в переменной `model_randomforest_class_weight`:

In [43]:
predictions_test = model_randomforest_class_weight.predict(data_test_features)
probabilities_valid = model_randomforest_class_weight.predict_proba(data_test_features)
probabilities_one_valid = probabilities_valid[:, 1]
print('F1-мера на тестовой выборке', f1_score(data_test_target, predictions_test))
print('AUC-ROC на тестовой выборке', roc_auc_score(data_test_target, probabilities_one_valid))

F1-мера на тестовой выборке 0.6042216358839051
AUC-ROC на тестовой выборке 0.8557333271229729


Как можно заметить у модели **Случайный лес** на тестовой выборке значение по метрике *F1*-мера не сильно меньше упало с **0.64** на валидационной выборке до **0.60**, что говорит о том, что мы сделали правильный выбор в пользу этой модели. Также по метрике *AUC-ROC* видно, что выбраная модель сильно отличается от случайной модели у которой значение по данной метрике равно **0.5**, что говорит о вменяемости выбранной модели.

### Общий вывод

В рамках задания нам нужно было построить модель машинного обучения, которая может предсказать уйдет ли клиента от банка в ближайшее время или нет на основании исторических данных о поведении клиентов, значение по метрике *F1*-мера должно составлять не менее **0.59**. 

Было обучено несколько моделей машинного обучения по предоставленным данным, а также использованы использованы различные методы балансировки классов и были получены следующие результаты:
- Модель **Дерево решений** значение метрики *F1*-мера без балансировки классов на валидационной выборке— **0.57**;
- Модель **Случайный лес** значение метрики *F1*-мера без балансировки классов на валидационной выборке— **0.58**;
- Модель **Логистическая регрессия** значение метрики *F1*-мера без балансировки классов на валидационной выборке— **0.30**;
- Модель **Дерево решений** значение метрики *F1*-мера с балансировкой классов с помощью аргумента `class_weight` на валидационной выборке— **0.57**;
- Модель **Случайный лес** значение метрики *F1*-мера с балансировкой классов с помощью аргумента `class_weight` на валидационной выборке— **0.64**;
- Модель **Логистическая регрессия** значение метрики *F1*-мера с балансировкой классов с помощью аргумента `class_weight` на валидационной выборке— **0.50**;
- Модель **Дерево решений** значение метрики *F1*-мера с балансировкой классов с помощью техники **upsampled** на валидационной выборке— **0.57**;
- Модель **Случайный лес** значение метрики *F1*-мера с балансировкой классов с помощью техники **upsampled** на валидационной выборке— **0.57**;
- Модель **Логистическая регрессия** значение метрики *F1*-мера с балансировкой классов с помощью техники **upsampled** на валидационной выборке— **0.50**;
- Модель **Дерево решений** значение метрики *F1*-мера с балансировкой классов с помощью техники **downsample** на валидационной выборке— **0.57**;
- Модель **Случайный лес** значение метрики *F1*-мера с балансировкой классов с помощью техники **downsample** на валидационной выборке— **0.58**;
- Модель **Логистическая регрессия** значение метрики *F1*-мера с балансировкой классов с помощью техники **downsample** на валидационной выборке— **0.50**;

На основании проведенной проверки обучения моделей на валидационной выборке была выбрана модель **Случайный лес** с гиперпараметрами `n_estimators` — количество деревьев в лесу равное **50** и `max_depth` — глубина дерева равное **9** при балансировке классов с помощью аргумента `class_weight`для проверки на тестовой выборке, на тестовой выборке данная модель показала значение метрики *F1*-мера равное **0.60**, что не сильно меньше, чем на валидационной выборке.

На основании выше перечисленного для прогноза ухода клиента от банка необходимо использовать модель **Случайный лес**, так как данная модель показала максимальную значение по метрике *F1*-мера **0.60** и это значение не сильно меньше, чем на валидационной выборке.